In [14]:
import os
from tqdm import tqdm
os.listdir('../weis/')

['cb']

In [ ]:
# Load configuration from environment variables
symbol = os.getenv('SYMBOL')
interval = int(os.getenv('INTERVAL'))
tss_n_splits = int(os.getenv('TSS_N_SPLITS'))
n_back_features = int(os.getenv('N_BACK_FEATURES'))
tss_test_size = int(os.getenv('TSS_TEST_SIZE'))

In [26]:
from predict import ModelPrediction
from datetime import datetime

def get_probas_for_bactesting(end_dt):

    prediction_model = ModelPrediction(
        model_path='../weis/cb',
        verbose=False
    )
    
    return prediction_model.full_prediction_cycle(symbol, interval, n_back_features, tss_n_splits, tss_test_size, end_date=end_dt)

In [27]:
from parsing.hist_parsing import KlineDataRetriever
from utils.utils import preprocess_data
from train import add_extrema_targets
from time import sleep
import zoneinfo

start_date = datetime(2024, 7, 27, 23, 0, tzinfo=zoneinfo.ZoneInfo('Europe/Moscow'))
end_date = datetime.now(zoneinfo.ZoneInfo('Europe/Moscow'))#datetime(2024, 7, 22, tzinfo=zoneinfo.ZoneInfo('Europe/Moscow'))

retriever = KlineDataRetriever('linear', symbol, interval, testnet=False)

backtest_data = retriever.fetch_last_data(start_date=start_date, end_date=end_date)

backtest_data = preprocess_data(backtest_data)
backtest_data = add_extrema_targets(backtest_data)

print(len(backtest_data))

200


In [28]:
preds = []


for dt in tqdm(backtest_data['DATETIME'].tolist()):
    bhs = get_probas_for_bactesting(dt)
    
    preds.append(bhs)
    
    sleep(0.01)
    
preds[:10]

  0%|          | 0/200 [00:00<?, ?it/s]

[(0.08562884102765218, 0.530087793225847, 0.3842833657465008),
 (0.7294710729718036, 0.25484873572748756, 0.015680191300708878),
 (0.607035193271255, 0.366546120278373, 0.026418686450371955),
 (0.7366844080198283, 0.24949003912670678, 0.013825552853464866),
 (0.746284065420078, 0.24133623664576717, 0.012379697934154827),
 (0.6736165991892107, 0.3063934554231988, 0.01998994538759054),
 (0.6516302891998411, 0.32537502966122894, 0.02299468113893007),
 (0.6804546335710056, 0.3015409365014792, 0.01800442992751532),
 (0.4648664175982106, 0.4899473921963919, 0.045186190205397506),
 (0.6233199252738211, 0.34956788842826647, 0.027112186297912448)]

In [29]:
buy_probas = [x[0] for x in preds]
sell_probas = [x[2] for x in preds]
hold_probas = [x[1] for x in preds]

ts = [0.65, 0, 0.7]

def make_decision(x, ts):
    b, h, s = x
    if b > ts[0]:
        return 0
    elif s > ts[2]:
        return 2
    else:
        return 1

decisions = [make_decision(x, ts) for x in preds]

In [24]:
import plotly.graph_objects as go
import numpy as np
from datetime import datetime

def plot_decision_probas(dates, closes, buy_probas, hold_probas, sell_probas):
    """
    Plots the close prices with buy, sell, and hold probabilities using Plotly.
    
    :param dates: List of datetime objects.
    :param closes: List of close prices.
    :param buy_probas: List of buy probabilities.
    :param hold_probas: List of hold probabilities.
    :param sell_probas: List of sell probabilities.
    """
    # Create a figure
    fig = go.Figure()

    # Plot close prices
    fig.add_trace(go.Scatter(x=dates, y=closes, mode='lines', name='Close Price', line=dict(color='gray')))

    # Plot buy probabilities
    fig.add_trace(go.Scatter(
        x=dates,
        y=closes,
        mode='markers',
        marker=dict(
            size=np.array(buy_probas) * 10,
            color='green',
            opacity=0.6,
            symbol='triangle-up'
        ),
        name='Buy Probability'
    ))

    # Plot hold probabilities
    fig.add_trace(go.Scatter(
        x=dates,
        y=closes,
        mode='markers',
        marker=dict(
            size=np.array(hold_probas) * 10,
            color='blue',
            opacity=0.6,
            symbol='circle'
        ),
        name='Hold Probability'
    ))

    # Plot sell probabilities
    fig.add_trace(go.Scatter(
        x=dates,
        y=closes,
        mode='markers',
        marker=dict(
            size=np.array(sell_probas) * 10,
            color='red',
            opacity=0.6,
            symbol='x'
        ),
        name='Sell Probability'
    ))

    # Update layout for better clarity
    fig.update_layout(
        title='Close Prices with Buy, Hold, and Sell Probabilities',
        xaxis_title='Datetime',
        yaxis_title='Close Price',
        legend_title='Legend',
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    fig.show()

plot_decision_probas(dates=backtest_data['DATETIME'], closes=backtest_data['CLOSE'], buy_probas=buy_probas, hold_probas=hold_probas, sell_probas=sell_probas)

In [25]:
import plotly.graph_objs as go
import pandas as pd

def plot_decisions_with_markers(decisions, datetimes, closes):
    # Create a DataFrame from the provided lists
    df = pd.DataFrame({
        'DateTime': pd.to_datetime(datetimes),
        'Close': closes,
        'Decision': decisions
    })
    
    # Map decisions to colors, labels, and symbols
    decision_colors = {0: 'green', 1: 'blue', 2: 'red'}
    decision_labels = {0: 'Buy', 1: 'Hold', 2: 'Sell'}
    decision_symbols = {0: 'triangle-up', 1: 'circle', 2: 'x'}
    
    # Create the base plot
    fig = go.Figure()
    
    # Add the Close price line
    fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Close'], mode='lines', name='Close', line=dict(color='black')))
    
    # Add decision markers with different shapes and colors
    for decision in decision_colors.keys():
        decision_df = df[df['Decision'] == decision]
        fig.add_trace(go.Scatter(
            x=decision_df['DateTime'],
            y=decision_df['Close'],
            mode='markers',
            name=decision_labels[decision],
            marker=dict(color=decision_colors[decision], size=7, symbol=decision_symbols[decision])
        ))
    
    # Update layout for better visualization
    fig.update_layout(
        title='Decisions on Close Price Over Time',
        xaxis_title='DateTime',
        yaxis_title='Close Price',
        legend_title='Decisions',
        template='plotly_white'
    )
    
    fig.show()
    
plot_decisions_with_markers(datetimes=backtest_data['DATETIME'], closes=backtest_data['CLOSE'], decisions=decisions)